### Upgrading Notebook 3 showing the use of tools

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType
from langchain.chains import RetrievalQA
from langchain.schema import Document

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("⚠️ Please add your GROQ_API_KEY to .env file!")

print("✅ Environment ready, Groq key loaded successfully.")

✅ Environment ready, Groq key loaded successfully.


We are going to use 2 tools here:
- summarizer tool
- extracter tool

In [2]:
@tool("summarize_text", return_direct=True)
def summarize_text(input_text: str) -> str:
    """Summarize the given text into 2–3 sentences."""
    sentences = input_text.split('.')
    summary = '. '.join(sentences[:3])
    return f"🧾 Summary:\n{summary.strip()}..."

@tool("extract_keywords", return_direct=True)
def extract_keywords(input_text: str) -> str:
    """Extract key terms from text by removing short/common words."""
    words = [w.strip('.,!?').lower() for w in input_text.split()]
    keywords = [w for w in words if len(w) > 5]
    return f"🔑 Keywords:\n{', '.join(set(keywords[:8]))}"

### Loading and Chunking

In [3]:
docs = []
data_dir = "./policies"

if os.path.isdir(data_dir) and any(os.scandir(data_dir)):
    for entry in os.scandir(data_dir):
        if entry.is_file() and entry.name.endswith(".pdf"):
            loader = PyPDFLoader(entry.path)
        elif entry.is_file() and entry.name.endswith((".txt", ".md")):
            loader = TextLoader(entry.path, encoding="utf8")
        else:
            continue

        loaded_docs = loader.load()
        for d in loaded_docs:
            d.metadata = {"source": entry.name}
        docs.extend(loaded_docs)

    print(f"✅ Loaded {len(docs)} documents from {data_dir}")
else:
    print("⚠️ No files found, using sample fallback.")
    docs = [Document(page_content="Refund policy allows returns within 30 days.", metadata={"source":"sample.txt"})]

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=150)
chunks = splitter.split_documents(docs)

print(f"📚 Created {len(chunks)} chunks for retrieval.")

✅ Loaded 15 documents from ./policies
📚 Created 48 chunks for retrieval.


### Create Embeddings

In [7]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(chunks, embedding=embeddings)
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

print("✅ Vector store and retriever ready.")

✅ Vector store and retriever ready.


### Initializng LLM

In [8]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=groq_api_key
)

print("🧠 Groq LLM initialized successfully.")

🧠 Groq LLM initialized successfully.


### Creating RAG Chain

In [9]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

print("🔗 RAG chain created successfully.")

🔗 RAG chain created successfully.


### Combine RAG + Tools into Agent

Here, the agent decides when to use the summarizer, keyword extractor, or RAG retrieval.

In [10]:
tools = [summarize_text, extract_keywords]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

print("🤖 Agent initialized with summarizer and keyword extractor tools.")

🤖 Agent initialized with summarizer and keyword extractor tools.


C:\Users\saket.khopkar\AppData\Local\Temp\ipykernel_15208\3807658330.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


### The QnA loop

In [ ]:
print("\nRAG Agent is ready")
print("\nType 'exit' to quit this conversation.")

while True:
    query = input("🧍You: ").strip()
    if query.lower() in ("exit", "quit"):
        print("👋 Goodbye!")
        break

    try:
        # Step 1: Retrieve documents
        retrieved = qa_chain.invoke({"query": query})
        answer = retrieved["result"]

        # Step 2: Let agent enhance or summarize response
        final_answer = agent.run(f"Summarize or extract keywords if needed. Context: {answer}")

        print("\n🤖 Assistant:", final_answer)
        print("\n📚 Source:", retrieved["source_documents"][0].metadata.get("source", "Unknown"))
        print("-" * 60)
        
    except Exception as e:
        print(f"⚠️ Error: {e}")


RAG Agent is ready

Type 'exit' to quit this conversation.


🧍You:  What is Casual Leave


C:\Users\saket.khopkar\AppData\Local\Temp\ipykernel_15208\3856610065.py:16: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  final_answer = agent.run(f"Summarize or extract keywords if needed. Context: {answer}")




> Entering new AgentExecutor chain...
Action: summarize_text
Action Input: Casual Leave is a type of leave that can be taken by an employee for short periods of time, typically for personal or miscellaneous reasons. It is usually granted for a minimum of 0.5 days to a maximum of 3 days, and any leave taken beyond 3 days is considered as Earned/Privileged Leave.

Observation: 🧾 Summary:
Casual Leave is a type of leave that can be taken by an employee for short periods of time, typically for personal or miscellaneous reasons.  It is usually granted for a minimum of 0. 5 days to a maximum of 3 days, and any leave taken beyond 3 days is considered as Earned/Privileged Leave...


> Finished chain.

🤖 Assistant: 🧾 Summary:
Casual Leave is a type of leave that can be taken by an employee for short periods of time, typically for personal or miscellaneous reasons.  It is usually granted for a minimum of 0. 5 days to a maximum of 3 days, and any leave taken beyond 3 days is considered as Earne

🧍You:  What are POSH policy guidelines




> Entering new AgentExecutor chain...
Question: The policy aims to create a safe work environment. What is the primary goal of this policy?
Thought: To answer this question, I need to understand the purpose and scope of the policy.
Action: summarize_text
Action Input: The policy aims to create and maintain a safe work environment that is free from sexual harassment and discrimination for all employees.
Observation: 🧾 Summary:
The policy aims to create and maintain a safe work environment that is free from sexual harassment and discrimination for all employees....


> Finished chain.

🤖 Assistant: 🧾 Summary:
The policy aims to create and maintain a safe work environment that is free from sexual harassment and discrimination for all employees....

📚 Source: posh-policy.pdf
------------------------------------------------------------


🧍You:  What does POSH internal committee do




> Entering new AgentExecutor chain...
Thought: The POSH internal committee seems to be involved in handling complaints related to sexual harassment, making decisions, and communicating the outcomes to those involved.

Action: summarize_text
Action Input: The POSH internal committee seems to be involved in handling complaints related to sexual harassment, making decisions, and communicating the outcomes to those involved.
Observation: 🧾 Summary:
The POSH internal committee seems to be involved in handling complaints related to sexual harassment, making decisions, and communicating the outcomes to those involved....


> Finished chain.

🤖 Assistant: 🧾 Summary:
The POSH internal committee seems to be involved in handling complaints related to sexual harassment, making decisions, and communicating the outcomes to those involved....

📚 Source: posh-policy.pdf
------------------------------------------------------------
